In [1]:
FIGURES_PATH = 'out/figures/'
DATASETS_PATH = 'out/datasets/'

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
# from tqdm import tqdm
from tqdm.notebook import tqdm
from multiprocesspandas import applyparallel
from pandarallel import pandarallel
import psutil


tqdm.pandas()
from helper import *

In [55]:
NROWS = None

In [56]:
data = pd.read_csv(DATASETS_PATH + 'data_processed.csv', nrows=NROWS).drop(columns=['Unnamed: 0'])
data['datetime'] = pd.to_datetime(data['datetime'])
# data

In [ ]:
def process_batch(x):
    ans = dict()
    for i in x['product_id'].values:
        if i in ans:
            ans[i] += 1
        else:
            ans[i] = 1
    return ans, x['product_id'].nunique()


def get_user_purchases(data):
    """
    :param data: receipts - pandas.DataFrame
    :return: ans: ans[i][j] = count of purchases by the user i of the product j - matrix
    """
    ans = dict()
    data = data[['gid', 'product_id']]
    
            
    pandarallel.initialize(progress_bar=True, use_memory_fs=True, nb_workers=psutil.cpu_count(logical=False))
    ans = data.groupby(by='gid').parallel_apply(process_batch)

    return ans.to_dict()

In [58]:
def delete_users_with_one(data, unique=False):
#     ans = dict()
#     data = data[['gid', 'product_id']]

            
    pandarallel.initialize(progress_bar=True, use_memory_fs=True, nb_workers=psutil.cpu_count(logical=False))
    if unique:
        ans = data.groupby(by='gid')[['gid', 'product_id']].parallel_apply(lambda x: x['product_id'].nunique())
    else:
        ans = data.groupby(by='gid')[['gid', 'product_id']].parallel_apply(lambda x: x.shape[0])
        
    ans = ans.loc[ans == 1].index.array
    
    if len(ans) != 0:
        ans = data.loc[~data['gid'].isin(ans)]
#         ans = data.drop(data.loc[data['gid']].index)
    else:
        ans = data

    return ans
    

In [59]:
data = delete_users_with_one(data, unique=True)

7485227
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


gid
0          3
1         46
2          1
3         25
4          4
          ..
593443     3
593444     8
593445     2
593446     1
593447     1
Length: 593448, dtype: int64
7395223


,gid,transaction_key,store_id,product_id,line_item_price,line_item_cost,line_type,datetime,category_id,weekday,line_quantity
0,0,0,2,0,2990.0,2516.10,0,2022-04-18 12:22:18,NaN,0,1.0
1,1,1,3,1,1413.0,1206.73,1,2022-08-26 18:00:29,NaN,4,2.0
2,1,1,3,2,338.0,179.51,1,2022-08-26 18:00:29,NaN,4,4.0
3,1,1,3,3,1634.0,1402.83,1,2022-08-26 18:00:29,NaN,4,1.0
4,1,1,3,4,78.0,61.70,1,2022-08-26 18:00:29,NaN,4,40.0
...,...,...,...,...,...,...,...,...,...,...,...
7485222,168014,1429117,3,512,125.0,66.02,0,2022-08-25 16:28:55,NaN,3,1.0
7485223,168014,1429117,3,16155,74.0,40.00,0,2022-08-25 16:28:55,NaN,3,1.0
7485224,508132,1429118,3,7240,280.0,180.06,3,2023-03-16 18:40:45,NaN,3,1.0
7485225,508132,1429118,3,198,250.0,NaN,3,2023-03-16 18:40:45,NaN,3,1.0


In [60]:
up = get_user_purchases(data)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


            gid  product_id
4472045  433329       31264
4472046  433329       60934
4472047  433329        2895
            gid  product_id
4472058  433331        8842
4472059  433331         234
4472060  433331       11876
            gid  product_id
4472087  433332        5792
4472088  433332        1333
4490074  433332       56225
4490075  433332       14953
4490076  433332        1304
4490077  433332         725
4490078  433332         586
4490079  433332       41997
4490080  433332       60791
4490081  433332        1921
4490082  433332       31613
            gid  product_id
4472128  433334        3530
4472129  433334         644
4732485  433334       36282
            gid  product_id
4472130  433335        7902
4472131  433335          51
4472132  433335       22026
4472133  433335       44457
4472134  433335        2603
            gid  product_id
4472147  433336       16595
4472148  433336       23572
4477862  433336       11412
4477863  433336       11433
4477864  433336     

7402058  433367         164
            gid  product_id
4472553  433368       16565
4472554  433368          51
4472555  433368       20380
4472556  433368       12044
4472557  433368         329
            gid  product_id
4472566  433369       10717
4472567  433369       11715
4472568  433369       22171
4472569  433369        6021
4472570  433369        5675
4472571  433369       18654
4761226  433369        2676
4761227  433369         188
            gid  product_id
4472586  433370       21737
4472587  433370       14222
5533291  433370       14222
5533292  433370       21737
            gid  product_id
4472652  433372        2794
4472653  433372        5287
4472654  433372       28079
4472655  433372        2792
4472656  433372       63508
4472657  433372        1640
            gid  product_id
4472692  433373        3239
4472693  433373        1294
4472694  433373        1295
            gid  product_id
4472695  433374       15520
4472696  433374        1837
4472697  433374     

7155230  433392        6010

            gid  product_id
3328447  356666        4937
3328448  356666       54053
            gid  product_id
4472986  433394       39993
4490981  433394       44081
4490982  433394       44082
4490983  433394       45296
            gid  product_id
3328453  356667       11433
3328454  356667        5726
3328455  356667       30057
3328456  356667         737
3328457  356667        3247
3328458  356667        1547            gid  product_id
4472998  433395         198
4472999  433395       10473
4473000  433395       40165

            gid  product_id
3328488  356668        5042
5001465  356668       18411
5001466  356668       23339
5001467  356668        6377
5001468  356668       30381
5001469  356668        1241
5001470  356668        3110
5001471  356668        9067
5001472  356668         911
5001473  356668       15704
5001474  356668       69931
5001475  356668         164
5001476  356668        6381
5845853  356668       19368
5845854  356668    

4961055  433411       18211
            gid  product_id
3328721  356685         296
3328722  356685       10243
3328723  356685        1746
3328724  356685       67619
3328725  356685       26371
            gid  product_id
4473206  433412       29570
4473207  433412       29276
4473208  433412       17298
4473209  433412       31161
4473210  433412       43916
            gid  product_id
3328739  356686       17368
3328740  356686       28490
3601625  356686        9913
3601626  356686       12917
3601627  356686        2435
4322319  356686       18086
4620913  356686        6007
            gid  product_id
4473220  433413       10548
4473221  433413        5914
4473222  433413        5890
4473223  433413       14227
            gid  product_id
3328776  356687        1805
3328777  356687       14491            gid  product_id
4473225  433415        3207
4473226  433415        3318
4473227  433415       12148
4473228  433415        1936

            gid  product_id
4473230  433416     

...

4172038  210191        5406


            gid  product_id
2382284  283067         243
2382285  283067        2548
2382286  283067       12053            gid  product_id
3341569  357615        5996
3341570  357615        1037
3341571  357615         310
3341572  357615         107
3341573  357615         177
3341574  357615       28680
3341575  357615          61
3341576  357615        1236
3341577  357615        2759
3341578  357615         605
3341579  357615       11422
3341580  357615        9697
3341581  357615         522
3341582  357615       15746
3341583  357615        5696
3341584  357615        2750
3359246  357615        1321
3359247  357615       40409
3359248  357615         464
3359249  357615        3346
3359250  357615       12995
3359251  357615        7353
3359252  357615        3344
3359253  357615       26809
3359254  357615        3255
6891256  357615       23733
6891257  357615         210
6891258  357615          51
6891259  357615       16835
6891260  357615    

7111542  138916        8055      gid  product_id
3632  763        2875
3633  763        2876
            gid  product_id
2382318  283070        6325
2382319  283070       45248
2382320  283070        8478           gid  product_id
414604   69199       18829
414605   69199       53394
414606   69199       14595
414607   69199       12688
414608   69199          51
2980469  69199          51
2980470  69199       18678
2980471  69199       21429
2980472  69199         955
2980473  69199       20453
2980474  69199       33528
2980475  69199        3095
3909250  69199        1350
3909251  69199       24489
3909252  69199        2902
3909253  69199        2084
3909254  69199        7056
3909255  69199        6715
3909256  69199          51
3909257  69199         945
3909258  69199        1204
3909259  69199        1990
7038654  69199       19265
7038655  69199        1314
7038656  69199       39541
7038657  69199        4466
7038658  69199       20822
7072523  69199       13461
7072524  6919

4733369  210195        2975


            gid  product_id
942127   138919        1445
942128   138919        3056
942129   138919         738
942130   138919       11703
1338448  138919        2730
1356338  138919        6395
1356339  138919          51
1356340  138919       28809
1356341  138919       35929
1356342  138919        2586
1356343  138919       27088
5155055  138919        2588
5155056  138919        4704
5155057  138919          51
5155058  138919        5871
5155059  138919        4494
6180817  138919        3844
6180818  138919       13316
6180819  138919       16222
6180820  138919        2941
6180821  138919        9195
6180822  138919        9197
6180823  138919         164
6180824  138919       16224
6180825  138919       19144
6180826  138919       37119
6180827  138919       17019      gid  product_id
3639  766         164
3640  766        2881
3641  766        2882
            gid  product_id
4487845  434366         198
4487846  434366       26175
4487847  434366

4487908  434369       25010            gid  product_id
1587821  210199       55409
1587822  210199       25611
1587823  210199       75722
1587824  210199        7907
1587825  210199        6462
1587826  210199        9936
1587827  210199       34556
1605892  210199       14288
1605893  210199       30466
3649009  210199        8485
5819197  210199       81283
5819198  210199       63748
5819199  210199       79373            gid  product_id
3341700  357625       16248
3341701  357625         164
3341702  357625         449
3341703  357625       10804
3341704  357625         690
3341705  357625       31060
3341706  357625        1334
3341707  357625         579
3341708  357625         935
3463566  357625         223
3463567  357625          51
3463568  357625          61
3463569  357625       35574
7115768  357625         292      gid  product_id
3647  769        2887
3648  769        2888
3649  769        2889           gid  product_id
942144  138923        1959
942145  138923        

5432471  434377       20155            gid  product_id
942186   138926       12671
942187   138926       10790
942188   138926        4980
942189   138926       43148
5789005  138926       63381
5789006  138926       32791
5789007  138926       34466
5789008  138926       73528
6029834  138926       34687
           gid  product_id
414672   69208       16248
414673   69208        4761
414674   69208       16776
414675   69208        1037
414676   69208         512
3758773  69208       16070
3758774  69208       14066
3758775  69208       18829
3852394  69208        7627
3852395  69208        9317
3852396  69208       24723
3852397  69208         512
3852398  69208        6641
4130463  69208       11539
4885105  69208        1505
4885106  69208        1804
6487779  69208       59365


In [44]:
up
# for key in up.keys():
#     val = up[key]
#     for k in val.keys():
#         v = val[k]
#         if v > 2:
#             print(key, val)

{0: ({0: 1}, 1),
 1: ({1: 1,
   2: 1,
   3: 1,
   4: 1,
   5: 1,
   6: 1,
   7: 1,
   8: 1,
   9: 1,
   10: 1,
   11: 1,
   12: 1,
   13: 1,
   14: 1,
   15: 1,
   16: 1,
   17: 1,
   18: 1,
   19: 1,
   20: 1,
   21: 1,
   22: 1,
   23: 1,
   24: 1,
   25: 1,
   26: 1,
   27: 1,
   28: 1,
   29: 1,
   30: 1,
   31: 1,
   32: 1,
   33: 1,
   34: 1,
   35: 1,
   36: 1,
   37: 1,
   10451: 1,
   593: 1,
   1549: 1,
   2467: 1,
   164: 1,
   6452: 1,
   5731: 1,
   5123: 1,
   2247: 1},
  46),
 2: ({38: 1}, 1),
 3: ({39: 1}, 1),
 4: ({40: 1, 41: 1, 42: 1, 43: 1}, 4),
 5: ({44: 1, 45: 1, 46: 1}, 3),
 6: ({47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1}, 8),
 7: ({55: 1, 56: 1}, 2),
 8: ({57: 1,
   58: 1,
   59: 1,
   60: 1,
   61: 1,
   62: 1,
   63: 1,
   64: 1,
   65: 1,
   66: 1,
   67: 1,
   68: 1},
  12),
 9: ({69: 1, 70: 1}, 2),
 10: ({71: 1}, 1),
 11: ({72: 1,
   172: 1,
   9043: 1,
   10458: 1,
   10459: 1,
   10460: 1,
   10461: 1,
   10462: 1,
   2082: 1},
  9),
 12: ({73

In [ ]:
data = data.drop(data.loc[])

In [8]:
def get_date_distances(data):
    """
    Считаем по каждому пользователю ближайшие (по модулю даты) покупки товаров.
    Усредняем значения по каждому пользователю

    :param data:
    :return:
    """

    # def concat_ans(ans):
        # return np.sum(ans['timedelta']), ans.shape[0]

    def data_splitting(interval):
        nonlocal data
        batches = []
        data = data.sort_values(by='datetime')
        start = data.iloc[0].at['datetime']
        end = data.iloc[-1].at['datetime']
        while start <= end:
            sub_end = start + timedelta(days=interval)
            batch = data.loc[data['datetime'] >= start].loc[data['datetime'] < sub_end]
            batches.append(batch)
            start = sub_end

        return batches


    def do_dataframe(temps):
        # t = temps.values

        # print(t.shape)
        # t = np.concatenate(t)#np.concatenate(t))
        # print(t.shape)
        ans = pd.DataFrame(data=temps, columns=['product_1', 'product_2', 'timedelta'])
        ans['count'] = pd.Series(data=[1 for _ in range(ans.shape[0])])
        return ans


    def fill_ans(x):
        product_date = x[['product_id', 'datetime']]

        temp = []

        for i1, r1 in product_date.iterrows():
            for i2, r2 in product_date.iterrows():
                if i1 != i2:
                    p1, p2 = r1['product_id'], r2['product_id']
                    timedelta = (r1['datetime'] - r2['datetime']).days

                    is_found = False
                    for t in temp:
                        if t[0] == p1 and t[1] == p2:
                            if abs(timedelta) < abs(t[2]):
                                t[2] = timedelta
                                is_found = True

                    if not is_found:
                        temp.append([p1, p2, timedelta])

        if len(temp) != 0:
            return np.array(temp)

    data = data[['gid', 'product_id', 'datetime']]
    data.loc[:, 'datetime'] = data['datetime'].dt.date

    batches = data_splitting(interval=14)

    pandarallel.initialize(progress_bar=True, use_memory_fs=True)
    temps = []
    for batch in tqdm(batches):
        grouped_by_user = batch.groupby(by='gid')
#         temp = grouped_by_user.parallel_apply(fill_ans)
        temp = grouped_by_user.apply_parallel(fill_ans, num_processes=multiprocessing.cpu_count())
        temp = temp.dropna()
        temp = np.concatenate(np.concatenate(temp.values))


        temps.append(temp)

    temps = np.concatenate(temps)

    #
    # # print(temps)
    ans = do_dataframe(temps)
    #
    # del temps
    #
    ans.groupby(by=['product_1', 'product_2']).progress_apply(np.sum)#parallel_apply(np.sum)
#     ans.groupby(by=['product_1', 'product_2']).apply_parallel(np.sum, num_processes=multiprocessing.cpu_count())
    #
    return ans





In [9]:
# dists = get_date_distances(data)

In [10]:
# dists.sort_values(by='timedelta', ascending=False)

In [11]:
# data.info()

In [12]:
BATCH_SIZE = 100_000

In [13]:
def get_date_distances_map(data):
    """
    Считаем по каждому пользователю ближайшие (по модулю даты) покупки товаров.
    Усредняем значения по каждому пользователю

    :param data:
    :return:
    """

    # def concat_ans(ans):
        # return np.sum(ans['timedelta']), ans.shape[0]
        
    ans = dict()
    
    

    def data_splitting(interval):
        nonlocal data
        batches = []
        data = data.sort_values(by='datetime')
        start = data.iloc[0].at['datetime']
        end = data.iloc[-1].at['datetime']
        while start <= end:
            sub_end = start + timedelta(days=interval)
            batch = data.loc[data['datetime'] >= start].loc[data['datetime'] < sub_end]
            batches.append(batch)
            start = sub_end

        return batches
    
    def data_splitting_by_count(count=50):
        
        return batches
#         df1 = df.sample(frac = 0.5, random_state = 200)


    def do_dataframe(temps):
        # t = temps.values

        # print(t.shape)
        # t = np.concatenate(t)#np.concatenate(t))
        # print(t.shape)
        ans = pd.DataFrame(data=temps, columns=['product_1', 'product_2', 'timedelta'])
        ans['count'] = pd.Series(data=[1 for _ in range(ans.shape[0])])
        return ans

    
    
    def fill_ans(x):
        product_date = x[['product_id', 'datetime']]

        temp = []

        for i1, r1 in product_date.iterrows():
            for i2, r2 in product_date.iterrows():
                if i1 != i2:
                    p1, p2 = r1['product_id'], r2['product_id']
                    timedelta = (r1['datetime'] - r2['datetime']).days

                    is_found = False
                    for t in temp:
                        if t[0] == p1 and t[1] == p2:
                            if abs(timedelta) < abs(t[2]):
                                t[2] = timedelta
                            is_found = True

                    if not is_found:
                        temp.append([p1, p2, timedelta])

        if len(temp) != 0:
            return np.array(temp)

    
    
    def concat_for_all_users(arrs):
        ans  = dict()
        for arr in arrs:
            if arr is not None:
                for one_arr in arr:
                    p1, p2, td = one_arr
#                     print(one_arr)
                    if (p1, p2) in ans:
                        ans[(p1, p2)].append(td)
#                         ans[(p1, p2)] = (ans[(p1, p2)][0] + td, ans[(p1, p2)][1] + 1)
                    else:
                        ans[(p1, p2)] = [td]
#                         ans[(p1, p2)] = (td, 1)
                        
        return ans
    
    
#     def get_average(ans):
        
        
        

    data = data[['gid', 'product_id', 'datetime']]
    data.loc[:, 'datetime'] = data['datetime'].dt.date

#     batches = data_splitting(interval=20)
    batches = np.array_split(data, data.shape[0] // BATCH_SIZE)
    pandarallel.initialize(progress_bar=True, use_memory_fs=True, nb_workers=psutil.cpu_count(logical=False))
    temps = []
    for batch in tqdm(batches):
        grouped_by_user = batch.groupby(by='gid')
        temp = grouped_by_user.parallel_apply(fill_ans)
#         temp = grouped_by_user.apply_parallel(fill_ans, num_processes=multiprocessing.cpu_count())
        temp = temp.dropna()
    
        temp = np.concatenate(temp.values)
#         temp = np.concatenate(np.concatenate(temp.values))

        temps.append(temp)
        
    ans = concat_for_all_users(temps)
    return ans





In [14]:
dists_map = get_date_distances_map(data)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


  0%|          | 0/74 [00:00<?, ?it/s]

In [15]:
print(type(data.sample(frac=50 / data.shape[0], random_state=100)))

<class 'pandas.core.frame.DataFrame'>


In [16]:
# dists_map

In [17]:
# ans = dict()

def process_batch(batch):
    res = dict()
    for key in tqdm(batch.keys()):
        arr = np.array(batch[key])
        res[key] = [arr.mean(), arr.shape[0], np.quantile(arr, 0.75) - np.quantile(arr, 0.25)]
    return res


def concat_batches(dist):
    ans.update(dist)
    

def transform_dists(dists):
    
    def chunks(dictionary, size):
        items = list(dictionary.items())
        return (dict(items[i : i + size]) for i in range(0, len(items), size))

    def custom_error_callback(error):
        print(f'Got an Error: {error}', flush=True)
    
    
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    
    for batch in chunks(dists, len(dists) // multiprocessing.cpu_count()):
        pool.apply_async(
            process_batch,
            args=(batch,),
            callback=concat_batches,
            error_callback=custom_error_callback,
        )
    
    pool.close()
    pool.join()
    
        
    return ans

In [ ]:
ans = dict()
trans_dists = transform_dists(dists_map)
del ans
trans_dists

In [ ]:
els = list(trans_dists.items())
els.sort(key=(lambda x: x[1][2]), reverse=True)
els
# for item in trans_dists.items():
#     if item[1][2] != 0:
#         print(item)

